In [ ]:
!pip install --upgrade google-api-python-client

https://ga-dev-tools.appspot.com/dimensions-metrics-explorer/?

In [1]:
import pandas as pd
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import seaborn as sns
from datetime import datetime
import pytz
import matplotlib.pyplot as plt

In [6]:
# setup
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_secrets.json'
VIEW_ID = 'Analytics TableID '
 
def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    return analytics

def response2df(response):
    report = response.get('reports', [])[0] # expected just one report
    # headers
    header_dimensions = report.get('columnHeader', {}).get('dimensions', [])
    header_metrics = [value['name'] for value in report.get('columnHeader', {}).get('metricHeader', {}).get('metricHeaderEntries', [])]
    headers = header_dimensions + header_metrics
    headers = list(map((lambda x: x.split(':', 1)[-1]), headers)) # removes "ga:" from each column
    # values
    values = []
    rows = report.get('data', {}).get('rows', [])
    for row in rows:
        values_dimensions = row.get('dimensions', [])
        values_metrics = row.get('metrics', [])[0].get('values', [])
        values.append(values_dimensions + values_metrics)
    # to dataframe
    df = pd.DataFrame(columns=headers, data=values)
    return df

In [7]:
def get_report(analytics):
    return analytics.reports().batchGet(body={'reportRequests': [{
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': '180daysAgo', 'endDate': 'today'}],
        'dimensions': [
            {'name': 'ga:pagePath'},
            {'name': 'ga:previousPagePath'},
            {'name': 'ga:sourceMedium'},
            {'name': 'ga:country'},
            {'name': 'ga:city'},
            {'name': 'ga:language'},
            {'name': 'ga:secondPagePath'},
            {'name': 'ga:daysSinceLastSession'},
            {'name': 'ga:dataSource'}
        ], 'metrics': [
            {'expression': 'ga:pageviews'},
            {'expression': 'ga:avgTimeOnPage'},
            {'expression': 'ga:uniquePageviews'}
        ], 
        'pageSize':10000
    }]}).execute()

analytics = initialize_analyticsreporting()
response = get_report(analytics)
df = response2df(response)